In [4]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime

In [5]:
envio_bq = pd.read_csv('conferenciaModelosAnos.csv', sep=',')
envio_bq['data_hora_atualizacao'] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

envio_bq

,Label,Value,cod_veiculo,Modelo,codigoModelo,Ano,codigoAno,data_hora_atualizacao
0,Acura,1,1,Integra GS 1.8,1,1992 Gasolina,1992-1,03/07/2023 11:37:04
1,Acura,1,1,Integra GS 1.8,1,1991 Gasolina,1991-1,03/07/2023 11:37:04
2,Acura,1,1,Legend 3.2/3.5,2,1998 Gasolina,1998-1,03/07/2023 11:37:04
3,Acura,1,1,Legend 3.2/3.5,2,1997 Gasolina,1997-1,03/07/2023 11:37:04
4,Acura,1,1,Legend 3.2/3.5,2,1996 Gasolina,1996-1,03/07/2023 11:37:04
...,...,...,...,...,...,...,...,...
45788,WALKBUS,144,3,Phanter Special Urbano/SPTRANS 2p (die.),4574,2012,2012-3,03/07/2023 11:37:04
45789,WALKBUS,144,3,Phanter Special Urbano/SPTRANS 2p (die.),4574,2011,2011-3,03/07/2023 11:37:04
45790,WALKBUS,144,3,Phanter Special Urbano/SPTRANS 2p (die.),4574,2010,2010-3,03/07/2023 11:37:04
45791,WALKBUS,144,3,Phanter Special Urbano/SPTRANS 2p (die.),4574,2009,2009-3,03/07/2023 11:37:04


In [3]:
envio_bq.dtypes

Label                    object
Value                     int64
cod_veiculo               int64
Modelo                   object
codigoModelo              int64
Ano                      object
codigoAno                object
data_hora_atualizacao    object
dtype: object

In [6]:
key_path = ""
credentials = service_account.Credentials.from_service_account_file(key_path)

In [7]:
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [8]:
project_id = 'project'
dataset_name = 'dataset'
table_name = 'table'

schema = [
    bigquery.SchemaField("Label", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("Value", "INT64", mode="NULLABLE"),
    bigquery.SchemaField("cod_veiculo", "INT64", mode="NULLABLE"),
    bigquery.SchemaField("Modelo", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("codigoModelo", "INT64", mode="NULLABLE"),
    bigquery.SchemaField("Ano", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("codigoAno", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("data_hora_atualizacao", "STRING", mode="NULLABLE")
]

In [9]:
table_ref = client.dataset(dataset_name).table(table_name)
try:
    client.get_table(table_ref)
    print('A tabela já existe. Os novos dados serão adicionados a ela.')
except:
    print('A tabela não existe. Uma nova tabela será criada.')
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    print('Tabela criada com sucesso.')

A tabela já existe. Os novos dados serão adicionados a ela.


In [10]:
job_config = bigquery.LoadJobConfig(schema=schema)
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

load_job = client.load_table_from_dataframe(envio_bq, table_ref, job_config=job_config)
print("Starting job {}".format(load_job.job_id))

load_job.result()
print("Job finished.")

destination_table = client.get_table(table_ref)
print("Loaded {} rows.".format(destination_table.num_rows))

Starting job ed66988f-6d73-4ac7-b8fe-197255313163
Job finished.
Loaded 45793 rows.
